# Retail Sales Analytics Dashboard

This notebook queries the transformed star schema data stored in Amazon S3 using Amazon Athena, and builds an interactive sales dashboard with filtering by country, product line, date range, and top-N products.

## Setup

Import required libraries.

In [ ]:
import pandas as pd
import awswrangler as wr
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import warnings

## Query Data from Athena

After the AWS Glue ETL job runs, the star schema is available in the Glue Data Catalog with the following structure:

![Star Schema](./images/schema_after_ETL.png)

- **`fact_orders`** — one row per order line, with measurements (quantity, price, order amount)
- **`dim_products`** — product catalog with product line descriptions
- **`dim_customers`** — customer details
- **`dim_locations`** — geographic context (city, state, country)

Set the Glue database name to match your deployed Terraform configuration:

In [ ]:
GLUE_DATABASE = "retail-analytics-db"  # Update this to match your deployed Glue catalog database name

products_df = wr.athena.read_sql_query(
    """
    SELECT * FROM dim_products
    """,
    database=GLUE_DATABASE,
)

products_df.head()

## Total Sales by Country

Aggregate total order amounts from `fact_orders` joined with `dim_locations`, grouped by country.

In [ ]:
# Retrieve total sales per country by joining 'fact_orders' with 'dim_locations' on postal code.
# The result is grouped by the 'country' column, and total sales is calculated as the sum of order amounts.
product_sales_by_country_df = wr.athena.read_sql_query(
    """
    SELECT
        dim_locations.country,
        SUM(fact_orders.orderAmount) AS total_sales
    FROM
        fact_orders
    JOIN
        dim_locations ON fact_orders.postalCode = dim_locations.postalCode
    GROUP BY 1
    """,
    database=GLUE_DATABASE,
)

product_sales_by_country_df.sort_values("total_sales", ascending=False).head(10)

## Detailed Sales by Product Line, Product, and Country

Join `fact_orders`, `dim_products`, and `dim_locations` to get granular sales data for the interactive dashboard.

In [ ]:
# Retrieve detailed sales information by joining 'fact_orders' with 'dim_products' and 'dim_locations'.
# Selects order date, product line, product name, country, and total sales (sum of order amounts).
# Grouped by order date, product line, product name, and country.
product_sales_df = wr.athena.read_sql_query(
    """
    SELECT
        fact_orders.orderDate,
        dim_products.productLine,
        dim_products.productName,
        dim_locations.country,
        SUM(fact_orders.orderAmount) AS total_sales
    FROM
        fact_orders
    JOIN
        dim_products ON fact_orders.productCode = dim_products.productCode
    JOIN
        dim_locations ON fact_orders.postalCode = dim_locations.postalCode
    GROUP BY 1, 2, 3, 4
    """,
    database=GLUE_DATABASE,
)

product_sales_df.head()

## Interactive Sales Dashboard

Use the dropdowns and sliders to filter by country, product line, date range, and top-N products.

In [ ]:
product_sales_df['orderdate'] = pd.to_datetime(product_sales_df['orderdate'])

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')

# Country Dropdown Widget
country_widget = widgets.Dropdown(
    options=["ALL"] + sorted(list(product_sales_df.country.unique())),
    value="ALL",
    description="Country",
)

# Product Line Dropdown Widget
productline_widget = widgets.Dropdown(
    options=["ALL"] + sorted(list(product_sales_df.productline.unique())),
    value="ALL",
    description="Product Line",
)

# Interactive Widgets for Date Range, Country, Product Line, and Top N
@widgets.interact(
    start_date=widgets.DatePicker(value=product_sales_df.orderdate.min(), description="Start Date"),
    end_date=widgets.DatePicker(value=product_sales_df.orderdate.max(), description="End Date"),
    country=country_widget,
    productline=productline_widget,
    top_n=widgets.IntSlider(value=5, min=1, max=10, step=1, description="Top N"),
)

# Function to Plot Top N Sales
def plot_top_n_sales(start_date, end_date, country, productline, top_n):

    # Filter data based on selected date range
    filtered_df = product_sales_df[
        (product_sales_df.orderdate >= pd.Timestamp(start_date))
        & (product_sales_df.orderdate <= pd.Timestamp(end_date))
    ]
    filtered_df = filtered_df.drop('orderdate', axis=1)
    filtered_df = filtered_df.groupby(['productline', 'productname', 'country']).sum().reset_index()

    # Build title string based on selected filters
    title_str = f"Top {top_n} Popular "

    if productline != "ALL":
        filtered_df = filtered_df[filtered_df.productline == productline]
        title_str += productline
    else:
        filtered_df = filtered_df.groupby(['productname', 'country']).sum().reset_index()
        title_str += "Products"

    if country != "ALL":
        filtered_df = filtered_df[filtered_df.country == country]
        title_str += " in " + country
    else:
        filtered_df = filtered_df.groupby(['productname']).sum().reset_index()

    # Plotting the bar chart
    if not (filtered_df.empty):
        try:
            ax = sns.barplot(
                x="total_sales",
                y="productname",
                data=filtered_df.sort_values("total_sales", ascending=False).head(top_n)
            )

            ax.set(
                xlabel="Total Sales",
                ylabel="Product Name",
                title=title_str
            )
        except:
            print("error")
    else:
        print(f"No sales of {productline} to {country} during that period")